In [13]:
import numpy as np
import pandas as pd

from typing import Callable, List, Tuple

In [ ]:
class Function:
    def __init__(self, func: Callable, x0: np.float16, x1: np.float16, epsilon: np.float16, maxit: int=20, derivative: Callable=None):
        """
        ARGS:
            func = Função analisada **(deve ser passada como função lambda)**
            x0 = Valor de x0 (menor)
            x1 = Valor de x1 (maior)
            epsilon = Tolerância de erro
            maxit = Máximo de iterações
            derivative (opcional) = Derivada da função **(essencial para o método de Newton)**
        """
        self.func = func
        self.x0 = x0
        self.x1 = x1
        self.epsilon = epsilon
        self.maxit = maxit
        self.derivative = derivative

    def _table(self, data: List[float], titles: List[str]) -> pd.DataFrame:
        """
        Cria a tabela com as informações de cada iteração

        ARGS:
            title = Título da tabela
            values = Valores da tabela
        """
        pd.options.display.float_format = '{:.5E}'.format
        return pd.DataFrame(data, columns=titles)
    
    def printFormated(df: pd.DataFrame) -> None:
        """
        Imprime a tabela segundo a formatação requisitada pelo exercício.

        ARGS:
            df = Dataframe gerado pelo método numérico
        """
        print(df.to_string(index=False, justify='center'))


    def bisect(self) -> Tuple[float, pd.DataFrame]:
        """
        Calcula a aproximação da raiz da função pelo método da bisseção.
        """
        # Variáveis iniciais
        a, b = self.x0, self.x1
        table_title = ['k', 'xk', 'f(xk)', 'step']
        table = []
        # Loop principal
        for i in range(self.maxit):
            step = abs(b - a)
            # Bisseção
            xk = (a + b) / 2
            f_xk = self.func(xk)
            table.append([i + 1, xk, f_xk, step])
            # Lógica de redução
            if (self.func(a) * f_xk < 0):
                b = xk
            else:
                a = xk

            if (step < self.epsilon):
                break
            
        full_table = self._table(table, table_title)
        return f_xk, full_table

    def newton(self) -> Tuple[float, pd.DataFrame]:
        """
        Calcula a aproximação da raiz da função pelo método de Newton.
        """
        # Verificação inicial, a fim de evitar erro de divisão por None
        if not callable(self.derivative):
            raise ValueError("A derivada da função (derivative) não foi fornecida ou não é uma função.")
        # Variáveis iniciais  
        xk = self.x0
        table_title = ['k', 'xk', 'f(xk)', 'f\'(xk)', 'step']
        table = []
        # Loop principal
        for i in range(self.maxit):
            _xk = xk
            f_xk = self.func(xk)
            f_dxk = self.derivative(xk)
            xk = xk - (f_xk / f_dxk) # x_{k+1}
            step = abs(xk - _xk)

            table.append([i + 1, xk, self.func(xk), self.derivative(xk), step])

        full_table = self._table(table, table_title)
        return self.func(xk), full_table

    def secant(self) -> Tuple[float, pd.DataFrame]:
        """
        Calcula a aproximação da raiz da função pelo método da secante.
        """
        # Variáveis iniciais
        xk0, xk1 = self.x0, self.x1
        table = []
        table_title = ['k', 'xk', 'f(xk)', 'step']
        # Loop principal
        for i in range(self.maxit):
            f_xk0 = self.func(xk0)
            f_xk1 = self.func(xk1)

            den = xk0 * f_xk1 - xk1 * f_xk0
            num = f_xk1 - f_xk0

            xk0 = xk1 # x_k se torna x_{k-1}
            xk1 = den / num # x_k se torna x_{k-1}
            step = (xk1 - xk0)

            table.append([i + 1, xk1, self.func(xk1), step])

        full_table = self._table(table, table_title)
        return self.func(xk1), full_table

In [ ]:
# Variáveis da entrada do método
# IMPORTANTE: SEMPRE DEFINA O TIPO DA VARIÁVEL COMO "FLOAT16"
f = lambda x: np.cos(x, dtype="float16") - np.sin(x, dtype="float16") # função
fd = lambda x: -1 * (np.sin(x, dtype="float16") + np.cos(x, dtype="float16")) # derivada da função
x0 = 0.0 # ponto inicial x0 ou extremidade a
x1 = np.pi / 2 # ponto inicial x1 ou extremidade b
eps = 0.01 # tolerância
maxit = 10 # máximo de iterações
func = Function(f, x0, x1, eps, maxit=maxit, derivative=fd)

# Aplica o método
app, df = func.bisect()

# Imprime a tabela
func.printFormated(df)
# Imprime a saída requisitada
print(f"\nSAIDA")
it = df.last_valid_index() + 1
print(f"x_{it} = {df["xk"][it - 1]}")
print(f"f(x_{it}) = {app}")
print(f"n = {it}")

 k      xk         f(xk)        step   
1  7.85398E-01  4.88281E-04 1.57080E+00
2  1.17810E+00 -5.41016E-01 7.85398E-01
3  9.81748E-01 -2.76367E-01 3.92699E-01
4  8.83573E-01 -1.38672E-01 1.96350E-01
5  8.34486E-01 -6.93359E-02 9.81748E-02
6  8.09942E-01 -3.46680E-02 4.90874E-02
7  7.97670E-01 -1.75781E-02 2.45437E-02
8  7.91534E-01 -8.78906E-03 1.22718E-02
9  7.88466E-01 -4.39453E-03 6.13592E-03

SAIDA
x_9 = 0.7884661249732196
f(x_9) = -0.00439453125
n = 9
